In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import itertools
import pickle
import matplotlib.dates as md


from pix_framework.statistics.distribution import DurationDistribution, get_best_fitting_distribution, DistributionType

import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [2]:
with open('../transformed_event_logs/PCR_start_end_train.pickle', 'rb') as f:
    train_data = pickle.load(f)

In [3]:
train_data

,case:concept:name,id:id_start,cpee:activity_start,cpee:instance_start,lifecycle:transition_start,cpee:lifecycle:transition_start,cpee:state_start,time:timestamp_start,data_start,cpee:description_start,...,seconds_in_day,day_of_week,Callback timeout,Export result,Export to EMS,Match patient data,Receive sample state,Send notification,Wait for plate validation,timeout
30928,10056,a2,a2,7bbdad33-b3e0-4e77-92a1-07d6f1429544,start,activity/calling,0.0,2023-04-03 12:02:52.198,"{'value': None, 'children': [('', {'value': ''...",0.0,...,43372,0,0,0,0,0,0,0,0,1
10998,10056,a6,a6,7bbdad33-b3e0-4e77-92a1-07d6f1429544,start,activity/calling,0.0,2023-04-03 12:02:52.201,"{'value': None, 'children': [('', {'value': ''...",0.0,...,43372,0,0,0,0,0,0,0,1,1
7606,10056,a4,a4,7bbdad33-b3e0-4e77-92a1-07d6f1429544,start,activity/calling,0.0,2023-04-03 12:02:52.207,"{'value': None, 'children': [('', {'value': ''...",0.0,...,43372,0,0,0,0,1,0,0,1,1
20701,10057,a2,a2,1887c54f-6403-420d-87ee-3cbc9ac6888a,start,activity/calling,0.0,2023-04-03 12:03:01.365,"{'value': None, 'children': [('', {'value': ''...",0.0,...,43381,0,0,0,0,0,0,0,0,1
1968,10057,a6,a6,1887c54f-6403-420d-87ee-3cbc9ac6888a,start,activity/calling,0.0,2023-04-03 12:03:01.369,"{'value': None, 'children': [('', {'value': ''...",0.0,...,43381,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,18121,a5,a5,e8075204-b208-4666-b262-91a222abd9b2,start,activity/calling,0.0,2023-06-12 21:08:04.345,"{'value': None, 'children': [('', {'value': ''...",0.0,...,76084,0,1,1,1,1,1,0,1,1
9978,18101,a5,a5,052a29f7-aa7a-4d86-a164-fc0f182f1cd7,start,activity/calling,0.0,2023-06-12 21:08:04.347,"{'value': None, 'children': [('', {'value': ''...",0.0,...,76084,0,1,1,1,1,1,0,1,1
11937,18108,a5,a5,0eae8ae9-f8cf-40fc-a5e3-eb78031df68a,start,activity/calling,0.0,2023-06-12 21:08:04.369,"{'value': None, 'children': [('', {'value': ''...",0.0,...,76084,0,1,1,1,1,1,0,1,1
31352,18109,a5,a5,c4c85eaa-560a-4a4e-8df0-38e3fa8a625c,start,activity/calling,0.0,2023-06-12 21:08:04.370,"{'value': None, 'children': [('', {'value': ''...",0.0,...,76084,0,1,1,1,1,1,0,1,1


In [4]:
from dumas_model import DumasModel

dm = DumasModel(train_data, types=['start'], resource=False)
dm.start_suspend_event_log = train_data
dm.set_up_models()

with open('../../../models/pcr_dumas/dumas_model.pickle', 'wb') as handle:
    pickle.dump(dm, handle)